<center>
<h2>Deep Learning using Linear Support Vector Machines</h2>
<p>by <i>Yichuan Tang</i></p>
<a href="http://arxiv.org/abs/1306.0239">http://arxiv.org/abs/1306.0239</a>
</center>
<p><b>Abstract: </b>Recently, fully-connected and convolutional neural networks have been trained to achieve state-of-the-art performance on a wide variety of tasks such as speech recognition, image classification, natural language processing, and bioinformatics. For classification tasks, most of these "deep learning" models employ the softmax activation function for prediction and minimize cross-entropy loss. In this paper, we demonstrate a small but consistent advantage of replacing the softmax layer with a linear support vector machine. Learning minimizes a margin-based loss instead of the cross-entropy loss. While there have been various combinations of neural nets and SVMs in prior art, our results using L2-SVMs show that by simply replacing softmax with linear SVMs gives significant gains on popular deep learning datasets MNIST, CIFAR-10, and the ICML 2013 Representation Learning Workshop's face expression recognition challenge.</p>

## Libraries

In [ ]:
import tensorflow as tf
from keras.datasets import mnist, cifar10
from keras.layers import Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.layers import GaussianNoise
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import numpy as np
from keras.models import Sequential
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator

## SVM Loss Function

The following loss fuction is based on the presented formula in [this course](https://cs231n.github.io/linear-classify/#multiclass-support-vector-machine-loss).

<center>
<img src="https://drive.google.com/uc?export=view&id=1ioDXTGPsD9vzbcVkBwQBOqa2CTw9gxa8"
</center>

In [ ]:
def svm_loss(layer, reg_weight=1, loss_weight=1):
    weights = layer.weights[0]
    weights_tf = tf.convert_to_tensor(weights)
    
    def squared_categorical_hinge_loss(y_true, y_pred):
        pos = K.sum(y_true * y_pred, axis=-1)
        neg = K.max((1.0 - y_true) * y_pred, axis=-1)
        hinge_loss = K.mean(K.square(K.maximum(0.0, 1.0 - pos + neg)), axis=-1)
        regularization_loss = tf.reduce_sum(tf.square(weights_tf))
        return reg_weight*regularization_loss + loss_weight*hinge_loss
    
    return squared_categorical_hinge_loss

# MNIST

## Dataset

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
num_classes = 10

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

60000 train samples
10000 test samples


## PCA

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scalar = StandardScaler()
scaled_x_train = scalar.fit_transform(x_train)
scaled_x_test = scalar.transform(x_test)

In [ ]:
pca = PCA(n_components = 70)
pca.fit(scaled_x_train)
x_train = pca.transform(scaled_x_train)
print('x_train pca shape:', x_train.shape)
x_test = pca.transform(scaled_x_test)
print('x_test pca shape:', x_test.shape)

x_train pca shape: (60000, 70)
x_test pca shape: (10000, 70)


## SVM Model

The original paper suggested a fully conneted model with the following specification:


*   Two hidden layer of 512 units
*   300 minibatches of 200 samples each
*   Stocastic gradiant descent wit momentum
*   Learning rate is linearly decayed from 0.1 to 0.0
*   A lot of Gaussian noise is added to the input. Noise of standard deviation of 1.0.

But the problems of the paper are:


*   It didn't specified the momentum value.
*   The batch size is better to be a power of two (like 256).
*   And most importantly it didn't specified the penalty value of hinge loss.





In [ ]:
model = Sequential()
model.add(GaussianNoise(1.0, input_shape=(x_train.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, use_bias=False, activation='tanh', name='svm'))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0005,
                                    decay=0.0005/400,
                                    momentum=0.9)
model.compile(optimizer = optimizer,
              loss = svm_loss(model.get_layer('svm'), 0.5, 0.5),
              metrics = ['accuracy'])

In [ ]:
batch_size = 256
epochs = 400

history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test))

Epoch 1/400
235/235 [==============================] - 1s 4ms/step - loss: 10.5507 - accuracy: 0.4717 - val_loss: 10.4739 - val_accuracy: 0.6456
Epoch 2/400
235/235 [==============================] - 1s 4ms/step - loss: 10.4820 - accuracy: 0.6026 - val_loss: 10.4665 - val_accuracy: 0.6991
Epoch 3/400
235/235 [==============================] - 1s 3ms/step - loss: 10.4768 - accuracy: 0.6477 - val_loss: 10.4558 - val_accuracy: 0.7318
Epoch 4/400
235/235 [==============================] - 1s 4ms/step - loss: 10.4681 - accuracy: 0.6811 - val_loss: 10.4297 - val_accuracy: 0.7628
Epoch 5/400
235/235 [==============================] - 1s 3ms/step - loss: 10.4383 - accuracy: 0.7214 - val_loss: 10.3545 - val_accuracy: 0.8103
Epoch 6/400
235/235 [==============================] - 1s 3ms/step - loss: 10.3277 - accuracy: 0.7877 - val_loss: 10.2008 - val_accuracy: 0.8761
Epoch 7/400
235/235 [==============================] - 1s 4ms/step - loss: 10.2357 - accuracy: 0.8503 - val_loss: 10.1576 - val_ac

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 10.02246379852295
Test accuracy: 0.9783999919891357


In [ ]:
print('Best validation accuracy:', max(history.history['val_accuracy']))

Best validation accuracy: 0.9785000085830688


## Softmax Model

In [ ]:
model = Sequential()
model.add(GaussianNoise(1.0, input_shape=(x_train.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(10, use_bias=False, activation='softmax', kernel_regularizer=l2(0.0001), name='softmax'))

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.0005,
                                    decay=0.0005/400,
                                    momentum=0.9)
model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
batch_size = 256
epochs = 400

history = model.fit(x_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test))

Epoch 1/400
235/235 [==============================] - 1s 4ms/step - loss: 1.1873 - accuracy: 0.6593 - val_loss: 0.5968 - val_accuracy: 0.8562
Epoch 2/400
235/235 [==============================] - 1s 3ms/step - loss: 0.5739 - accuracy: 0.8416 - val_loss: 0.4230 - val_accuracy: 0.8888
Epoch 3/400
235/235 [==============================] - 1s 3ms/step - loss: 0.4655 - accuracy: 0.8660 - val_loss: 0.3587 - val_accuracy: 0.9024
Epoch 4/400
235/235 [==============================] - 1s 3ms/step - loss: 0.4147 - accuracy: 0.8766 - val_loss: 0.3222 - val_accuracy: 0.9098
Epoch 5/400
235/235 [==============================] - 1s 4ms/step - loss: 0.3822 - accuracy: 0.8857 - val_loss: 0.2978 - val_accuracy: 0.9151
Epoch 6/400
235/235 [==============================] - 1s 4ms/step - loss: 0.3596 - accuracy: 0.8927 - val_loss: 0.2800 - val_accuracy: 0.9202
Epoch 7/400
235/235 [==============================] - 1s 5ms/step - loss: 0.3442 - accuracy: 0.8963 - val_loss: 0.2653 - val_accuracy: 0.9230

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07764852046966553
Test accuracy: 0.9782000184059143


In [ ]:
print('Best validation accuracy:', max(history.history['val_accuracy']))

Best validation accuracy: 0.9785000085830688


# CIFAR

## Dataset

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
num_classes = 10

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


## Data Augmentation

In [ ]:
datagen = ImageDataGenerator(#zoom_range=0.1,
                             #width_shift_range=0.1,
                             #height_shift_range=0.1,
                             horizontal_flip=True)
it_train = datagen.flow(x_train, y_train)

## SVM Model

The original paper suggested a convolutional network with the following specification:


*   A Convolutional Network consisting of two convolutional layer
*   The first convolutional layer had 32 5 × 5 filters with ReLU activation function
*   The second convolutional layer had 64 5 × 5 filters with ReLU activation function
*   Both pooling layers used max pooling and downsampled by a factor of 2
*   The penultimate layer has 3072 hidden nodes and uses ReLU activation with a dropout rate of 0.2
*   Horizontal reflection and jitter is applied to the data randomly
Minibatch of 128 data cases


But the problems of the paper are:


*   It didn’t specified the optimization algorithm, learning rate, etc.
*   It didn’t say anything about the number of epochs.
*   And most importantly it didn’t specified the regularization weight value (𝜆).






In [ ]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(32, 32, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(3072, activation='relu', name='penultimate'))
model.add(Dropout(0.2))
model.add(Dense(10, use_bias=False, activation='tanh', name='svm'))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer = optimizer,
              loss = svm_loss(model.get_layer('svm'), 0.2, 0.8),
              metrics = ['accuracy'])

In [ ]:
batch_size = 128
epochs = 50

history = model.fit(it_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test))

Epoch 1/50
1563/1563 [==============================] - 13s 8ms/step - loss: 4.7940 - accuracy: 0.4027 - val_loss: 4.7685 - val_accuracy: 0.5055
Epoch 2/50
1563/1563 [==============================] - 12s 7ms/step - loss: 4.7119 - accuracy: 0.5810 - val_loss: 4.6663 - val_accuracy: 0.6137
Epoch 3/50
1563/1563 [==============================] - 11s 7ms/step - loss: 4.6188 - accuracy: 0.6706 - val_loss: 4.6067 - val_accuracy: 0.6775
Epoch 4/50
1563/1563 [==============================] - 12s 8ms/step - loss: 4.5565 - accuracy: 0.7194 - val_loss: 4.5771 - val_accuracy: 0.6961
Epoch 5/50
1563/1563 [==============================] - 12s 8ms/step - loss: 4.5048 - accuracy: 0.7543 - val_loss: 4.5535 - val_accuracy: 0.7140
Epoch 6/50
1563/1563 [==============================] - 12s 7ms/step - loss: 4.4586 - accuracy: 0.7867 - val_loss: 4.5309 - val_accuracy: 0.7345
Epoch 7/50
1563/1563 [==============================] - 12s 8ms/step - loss: 4.4173 - accuracy: 0.8119 - val_loss: 4.5295 - val_ac

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 4.900073528289795
Test accuracy: 0.7702000141143799


In [ ]:
print('Best validation accuracy:', max(history.history['val_accuracy']))

Best validation accuracy: 0.7730000019073486


## Softmax Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(32, 32, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(3072, activation='relu', name='penultimate'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
batch_size = 128
epochs = 50

history = model.fit(it_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (x_test, y_test))

Epoch 1/50
1563/1563 [==============================] - 12s 7ms/step - loss: 1.5164 - accuracy: 0.4576 - val_loss: 1.2396 - val_accuracy: 0.5631
Epoch 2/50
1563/1563 [==============================] - 12s 7ms/step - loss: 1.1866 - accuracy: 0.5825 - val_loss: 1.1483 - val_accuracy: 0.5866
Epoch 3/50
1563/1563 [==============================] - 11s 7ms/step - loss: 1.0462 - accuracy: 0.6329 - val_loss: 0.9925 - val_accuracy: 0.6525
Epoch 4/50
1563/1563 [==============================] - 11s 7ms/step - loss: 0.9577 - accuracy: 0.6643 - val_loss: 0.9550 - val_accuracy: 0.6669
Epoch 5/50
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8864 - accuracy: 0.6908 - val_loss: 0.9107 - val_accuracy: 0.6797
Epoch 6/50
1563/1563 [==============================] - 11s 7ms/step - loss: 0.8253 - accuracy: 0.7142 - val_loss: 0.9020 - val_accuracy: 0.6860
Epoch 7/50
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7732 - accuracy: 0.7306 - val_loss: 0.8440 - val_ac

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.2834279537200928
Test accuracy: 0.7638000249862671


In [ ]:
print('Best validation accuracy:', max(history.history['val_accuracy']))

Best validation accuracy: 0.7723000049591064
